In [ ]:
import sawty
import time

CHOICES = ["YES", "NO", "ABSTAIN"]

VOTER_CHOICES = {
    "Aya": "YES",
    "Bassem": "NO",
    "Charlie": "ABSTAIN",
    "Dana": "NO",
    "Ella": "NO",
}

QKD_KEY_LENGTH = 32

In [ ]:
voters = [Voter(name, qkd_key_length=QKD_KEY_LENGTH) for name in VOTER_CHOICES.keys()]
voter_dict = {voter.id: voter.public_key for voter in voters}

auth = Authenticator("AuthNode", voter_dict, qkd_key_length=QKD_KEY_LENGTH)

tally1 = Tallier("TallyNode1", qkd_key_length=QKD_KEY_LENGTH)
tally2 = Tallier("TallyNode2", qkd_key_length=QKD_KEY_LENGTH)
tally3 = Tallier("TallyNode3", qkd_key_length=QKD_KEY_LENGTH)
talliers = [tally1, tally2, tally3]

blockchain = Blockchain(talliers)

In [ ]:
for voter in voters:
    voter.establish_qkd_channel(auth)
    for tally in talliers:
        voter.establish_qkd_channel(tally)

for tally in talliers:
    auth.establish_qkd_channel(tally)

In [ ]:
for voter in voters:
    voter.sign_identity()
    voter_choice = VOTER_CHOICES[voter.name]
    vote_index = CHOICES.index(voter_choice)

    vote_payload = voter.send_vote(vote_index, auth, talliers[0])

    signature_hex, forwarded_vote = auth.forward_vote(vote_payload, talliers[0], voter)

    for tally in talliers:
        tally_vote = tally.receive_vote(forwarded_vote, auth)
        tally.add_vote(signature_hex, tally_vote)

In [ ]:
first_tallier = talliers[0]
votes_snapshot = first_tallier.received_votes.copy()
standard_timestamp = time.time()

proposed_blocks = []
for tally in talliers:
    block = blockchain.propose_block(
        votes_snapshot,
        tally,
        fixed_creator="TallyConsensus",
        fixed_timestamp=standard_timestamp
    )
    proposed_blocks.append(block)

if blockchain.byzantine_agreement(proposed_blocks):
    blockchain.display_chain()
    blockchain.save_json("blockchain.json")
    blockchain.simple_majority()